# Plasma dispersion relation: linear growth and damping rate

# Landau Damping
The complex frequency is described as $\omega = \omega_{r} + i \gamma$, where $\gamma$ is the growth or damping rate.

Following Peter Gary's *Theory of space plasma microinstabilities* (2009) Ch (2), the dispersion relation is given by Eq. (2.2.8), i.e.
$$
1+ \sum_{j} K_{j}(k, \omega) = 0
$$
where $K_{j}(k, w) = \frac{-k_{j}^{2}}{2k^2} Z'(\xi_{j})$, $Z(\xi_{j})$ is the plasma dispersion function, and $\xi_{j} = \frac{\omega}{\sqrt{2}kv_{j}}$. Note that $Z'(\xi_{j}) = -2[1+\xi_{j} Z(\xi_{j})]$, resulting in 
$$
1  +\sum_{j} \frac{k_{j}^{2}}{k^2} [1+\xi_{j} Z(\xi_{j})] = 0.
$$
The plasma dispersion function is defined as 
$$
Z(\xi) = \frac{1}{\pi} \int_{-\infty}^{\infty} \frac{dx \exp(-x^2)}{x-\xi} = e^{-\xi^2} \left( \frac{i}{\sqrt{\pi}} -2\int_{0}^{\xi} e^{x^{2}} dx\right) = i \sqrt{\pi} w(\xi)
$$
where $w(\xi)$ is the Faddeeva function.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import wofz 
import scipy

In [2]:
def Z_fun(z):
    return 1j*np.sqrt(np.pi)*wofz(z)

def Z_p_fun(z):
    return -2.0*(1+z*Z_fun(z))

def Z_pp_fun(z):
    return -2.0*(Z_fun(z) + z*Z_p_fun(z))

In [3]:
# Despersion relation
def disp_k(alpha=1):
    return lambda omega : 1 -  0.5*Z_p_fun(omega/(np.sqrt(2)*alpha))

# Despersion relation derivative [by omega]
def disp_p_k(alpha=1):
    return lambda omega : -  0.5*Z_pp_fun(omega/(np.sqrt(2)*alpha))/(np.sqrt(2)*alpha)

In [4]:
def newton(guess, f, df, rtol=1e-8, atol=1e-16, maxit=10000):
    
    it = 1
    prev_x = guess
    next_x = prev_x - f(prev_x)/df(prev_x)
    
    # Another possible condition
    #  abs( f(next_x) )>rtol*abs( f(guess) )+atol
    while( abs(next_x-prev_x)>rtol*abs(guess)+atol and it<maxit ):
        prev_x = next_x
        next_x = prev_x - f(prev_x)/df(prev_x)
        it = it+1
        
    return next_x,it

In [5]:
sol, it = newton(guess=1.0-0.6j, f=disp_k(alpha=1), df=disp_p_k(alpha=1))
print("w(k=1) = ", sol)
print("iterations for convergence = ", it)
print("dispersion satisfied = ", disp_k(alpha=1)(sol))

w(k=1) =  (2.0459048656906247-0.8513304586920561j)
iterations for convergence =  12
dispersion satisfied =  3.3306690738754696e-16j


linear Landau damping most simplied equation

In [6]:
def func(omega):
    zeta =  omega/np.sqrt(2)
    return 2  + zeta*Z_fun(zeta)

In [7]:
sol = scipy.optimize.newton(func, 2.2-0.5j, maxiter=10000)
print(sol)
func(sol)

(2.0459048656906242-0.8513304586920561j)


(-4.440892098500626e-16+5.551115123125783e-16j)

# Two-stream instability 

The complex frequency is described as $\omega = \omega_{r} + i \gamma$, where $\gamma$ is the growth or damping rate.
The zeroth-order distributions for the two species as drifting Maxwellians:
$$
f_{j}^{0}(v) = \frac{n_{j}}{(2\pi v_{j}^2)^{3/2}}\exp\left(-\frac{-(v-u_{j})^2}{2v_{j}^{2}} \right)
$$

Following Peter Gary's *Theory of space plasma microinstabilities* (2009), the dispersion relation is given by 

$$
1+ \sum_{j} K_{j}(k, \omega) = 0
$$

where $K_{j}(k, w) = \frac{-k_{j}^{2}}{2k^2} Z'(\xi_{j})$, $Z(\xi_{j})$ is the plasma dispersion function, and $\xi_{j} = \frac{\omega - ku_{j}}{\sqrt{2}kv_{j}}$. Note that $Z(\xi_{j}) = -2[1+\xi_{j} Z(\xi_{j})]$, resulting in 
$$
1  +\sum_{j} \frac{k_{j}^{2}}{k^2} [1+\xi_{j} Z(\xi_{j})] = 0.
$$
The plasma dispersion function is defined as 
$$
Z(\xi) = \frac{1}{\sqrt{\pi}} \int_{-\infty}^{\infty} \frac{dx \exp(-x^2)}{x-\xi} = e^{-\xi^2} \left( \frac{i}{\sqrt{\pi}} -2\int_{0}^{\xi} e^{x^{2}} dx\right) = i \sqrt{\pi} w(\xi)
$$
where $w(\xi)$ is the Faddeeva function.

In [8]:
# Despersion relation
def disp_k(alpha=1, u_1=1, u_2=-1):
    return lambda omega : 1 - 0.25*Z_p_fun((omega-u_1)/(np.sqrt(2)*alpha))/(alpha**2)- 0.25*Z_p_fun((omega-u_2)/(np.sqrt(2)*alpha))/(alpha**2)

# Despersion relation derivative [by omega]
def disp_p_k(alpha=1, u_1=1, u_2=-1):
    return lambda omega : -  0.5*(0.5*Z_pp_fun((omega-u_1)/(np.sqrt(2)*alpha))/(np.sqrt(2)*(alpha**3))+ 0.5*Z_pp_fun((omega-u_2)/(np.sqrt(2)*alpha))/((alpha**3)*np.sqrt(2)))

In [9]:
sol, it = newton(guess=0+0.185j, f=disp_k(alpha=1/(2*np.sqrt(2))), df=disp_p_k(alpha=1/(2*np.sqrt(2))))
print("w(k=1) = ", sol)
print("iterations for convergence = ", it)
print("dispersion satisfied = ", disp_k(alpha=1/(2*np.sqrt(2)))(sol))

w(k=1) =  0.18486005593754073j
iterations for convergence =  3
dispersion satisfied =  (1.5543122344752192e-15+0j)


two-stream instability most simplified equation

In [10]:
def func(omega):
    u_1=1
    u_2=-1
    zeta_1 = (omega-u_1)*2
    zeta_2 = (omega-u_2)*2
    return 9  + 4*zeta_1*Z_fun(zeta_1) + 4*zeta_2*Z_fun(zeta_2) 

In [11]:
sol = scipy.optimize.newton(func, 0+0.1j, maxiter=10000)
print(sol)
func(sol)

(-8.557817587084934e-17+0.18486005593754054j)


(4.440892098500626e-15+1.1102230246251565e-16j)

# Ion Acoustic Current Driven Instability 

The complex frequency is described as $\omega = \omega_{r} + i \gamma$, where $\gamma$ is the growth or damping rate.
The zeroth-order distributions for the two species as drifting Maxwellians:
$$
f_{j}^{0}(v) = \frac{n_{j}}{(2\pi v_{j}^2)^{3/2}}\exp\left(-\frac{-(v-u_{j})^2}{2v_{j}^{2}} \right)
$$

Following Peter Gary's *Theory of space plasma microinstabilities* (2009), the dispersion relation is given by 

$$
1+ \sum_{j} K_{j}(k, \omega) = 0
$$

where $K_{j}(k, w) = \frac{-k_{j}^{2}}{2k^2} Z'(\xi_{j})$, $Z(\xi_{j})$ is the plasma dispersion function, and $\xi_{j} = \frac{\omega - ku_{j}}{\sqrt{2}kv_{j}}$. Note that $Z(\xi_{j}) = -2[1+\xi_{j} Z(\xi_{j})]$, resulting in 
$$
1  +\sum_{j} \frac{k_{j}^{2}}{k^2} [1+\xi_{j} Z(\xi_{j})] = 0.
$$
The plasma dispersion function is defined as 
$$
Z(\xi) = \frac{1}{\sqrt{\pi}} \int_{-\infty}^{\infty} \frac{dx \exp(-x^2)}{x-\xi} = e^{-\xi^2} \left( \frac{i}{\sqrt{\pi}} -2\int_{0}^{\xi} e^{x^{2}} dx\right) = i \sqrt{\pi} w(\xi)
$$
where $w(\xi)$ is the Faddeeva function.

In [37]:
# Despersion relation
def disp_k(alpha_1=1, alpha_2=1, u_1=1, u_2=0):
    return lambda omega : 1 - 0.5*Z_p_fun((omega-u_1)/(np.sqrt(2)*alpha_1))/(alpha_1**2)- 0.5*Z_p_fun((omega-u_2)/(np.sqrt(2)*alpha_2))/(alpha_2**2)

# Despersion relation derivative [by omega]
def disp_p_k(alpha_1=1, alpha_2=1, u_1=1, u_2=0):
    return lambda omega : -  0.5*Z_pp_fun((omega-u_1)/(np.sqrt(2)*alpha_1))/(np.sqrt(2)*(alpha_1**3))- 0.5*Z_pp_fun((omega-u_2)/(np.sqrt(2)*alpha_2))/((alpha_2**3)*np.sqrt(2))

In [48]:
sol, it = newton(guess=1+0.2j, f=disp_k(alpha_1=1, alpha_2=np.sqrt(1/np.sqrt(50)), u_1=3), df=disp_p_k(alpha_1=1, alpha_2=np.sqrt(1/np.sqrt(50)), u_1=3))
print("w(k=1) = ", sol)
print("iterations for convergence = ", it)
print("dispersion satisfied = ", disp_k(alpha_1=1, alpha_2=np.sqrt(1/np.sqrt(50)), u_1=3)(sol))

w(k=1) =  (1.2162699232779275+0.15004756564704408j)
iterations for convergence =  5
dispersion satisfied =  (-5.218048215738236e-15+7.216449660063518e-16j)


In [55]:
sol, it = newton(guess=1+0.2j, f=disp_k(alpha_1=1, alpha_2=np.sqrt(1/50), u_1=3), df=disp_p_k(alpha_1=1, alpha_2=np.sqrt(1/50), u_1=3))
print("w(k=1) = ", sol)
print("iterations for convergence = ", it)
print("dispersion satisfied = ", disp_k(alpha_1=1, alpha_2=np.sqrt(1/50), u_1=3)(sol))

w(k=1) =  (1.068145703872401+0.20314184452154455j)
iterations for convergence =  4
dispersion satisfied =  (-9.325873406851315e-15-7.049916206369744e-15j)
